# SPR 2026 - Qwen 3 4B Few-shot BI-RADS

**Modelo:** Qwen/Qwen3-4B-Instruct

**Características:**
- Few-shot learning com 1 exemplo por categoria
- Prompt refinado com exemplos reais
- 4B parâmetros - cabe em T4

**Hipótese:** Exemplos concretos melhoram a classificação vs zero-shot

---
## CONFIGURAÇÃO KAGGLE:
1. **Add Input** → **Models** → `qwen3-4b-instruct`
2. **Add Input** → **Competition** → `spr-2026-mammography-report-classification`
3. **Settings** → Internet → **OFF**, GPU → **T4 x2**
---

In [ ]:
# ===== QWEN 3 4B FEW-SHOT =====

import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - Qwen 3 4B Few-shot BI-RADS")
print("="*60)

SEED = 42
MAX_NEW_TOKENS = 10
DATA_DIR = '/kaggle/input/competitions/spr-2026-mammography-report-classification'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.manual_seed(SEED)
np.random.seed(SEED)

def find_model_path():
    base = '/kaggle/input'
    def search_dir(directory, depth=0, max_depth=10):
        if depth > max_depth: return None
        try:
            for item in os.listdir(directory):
                path = os.path.join(directory, item)
                if os.path.isdir(path) and os.path.exists(os.path.join(path, 'config.json')):
                    return path
                result = search_dir(path, depth + 1, max_depth) if os.path.isdir(path) else None
                if result: return result
        except: pass
        return None
    return search_dir(base)

MODEL_PATH = find_model_path()
print(f"Device: {device}")
print(f"Model: {MODEL_PATH}")

In [ ]:
# ===== FEW-SHOT EXAMPLES =====

FEW_SHOT_EXAMPLES = """
## Exemplos:

Relatório: "Exame realizado para controle. Imagens mostram parênquima mamário denso, sem nódulos, calcificações suspeitas ou distorções arquiteturais. Pele e subcutâneo preservados. Axilas livres."
BI-RADS: 1

Relatório: "Presença de nódulo oval, circunscrito, paralelo à pele, no QSE da mama direita, medindo 8mm, com características benignas. Sugere-se fibroadenoma."
BI-RADS: 2

Relatório: "Identificado nódulo sólido, oval, circunscrito, de 12mm no QIE esquerdo. Provavelmente benigno. Recomenda-se controle em 6 meses."
BI-RADS: 3

Relatório: "Nódulo irregular, de contornos microlobulados, com 15mm na JQQ da mama direita. Apresenta algumas microcalcificações agrupadas. Biópsia recomendada."
BI-RADS: 4

Relatório: "Lesão espiculada, densa, de 25mm na região retroareolar esquerda, associada a microcalcificações pleomórficas agrupadas. Altamente sugestiva de malignidade."
BI-RADS: 5

Relatório: "Paciente com carcinoma ductal invasivo confirmado por biópsia prévia. Estadiamento pré-operatório."
BI-RADS: 6

Relatório: "Estudo prejudicado por compressão incompleta. Necessária complementação com incidências adicionais."
BI-RADS: 0
"""

SYSTEM_PROMPT = f"""Você é um radiologista especialista em classificação BI-RADS de mamografias.

## Categorias BI-RADS:
- 0: Incompleto - necessita imagens adicionais
- 1: Negativo - mamografia normal
- 2: Benigno - achados definitivamente benignos
- 3: Provavelmente benigno - <2% malignidade, seguimento 6 meses
- 4: Suspeito - 2-95% malignidade, biópsia recomendada
- 5: Altamente sugestivo de malignidade - >95%
- 6: Malignidade comprovada por biópsia

{FEW_SHOT_EXAMPLES}

Responda APENAS com o número da categoria (0-6)."""

USER_TEMPLATE = """Relatório:
{report}

BI-RADS:"""

In [ ]:
# ===== CARREGAR MODELO =====
print("Carregando modelo...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH, local_files_only=True,
    torch_dtype=torch.bfloat16, device_map="auto", low_cpu_mem_usage=True
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Modelo: {model.config.architectures}")

In [ ]:
# ===== DADOS =====
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"Train: {len(train_df)}, Test: {len(test_df)}")

In [ ]:
# ===== CLASSIFICAÇÃO =====
def classify_report(report, model, tokenizer):
    messages = [{"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": USER_TEMPLATE.format(report=report)}]
    
    if hasattr(tokenizer, 'apply_chat_template'):
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    else:
        text = f"{SYSTEM_PROMPT}\n\n{USER_TEMPLATE.format(report=report)}"
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=4096)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=MAX_NEW_TOKENS, do_sample=False,
                                  pad_token_id=tokenizer.pad_token_id, eos_token_id=tokenizer.eos_token_id)
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    for char in response.strip():
        if char.isdigit() and char in '0123456':
            return int(char)
    return 2

# Teste
sample = train_df.iloc[0]
print(f"Real: {sample['target']}, Pred: {classify_report(sample['report'], model, tokenizer)}")

In [ ]:
# ===== VALIDAÇÃO =====
from sklearn.metrics import f1_score, classification_report

val_sample = train_df.groupby('target', group_keys=False).apply(lambda x: x.sample(min(20, len(x)), random_state=SEED))
val_preds = [classify_report(row['report'], model, tokenizer) for _, row in tqdm(val_sample.iterrows(), total=len(val_sample))]
val_labels = val_sample['target'].values

print(f"\nF1-Macro: {f1_score(val_labels, val_preds, average='macro'):.5f}")
print(classification_report(val_labels, val_preds))

In [ ]:
# ===== SUBMISSION =====
test_preds = [classify_report(row['report'], model, tokenizer) for _, row in tqdm(test_df.iterrows(), total=len(test_df))]

submission = pd.DataFrame({'ID': test_df['ID'], 'target': test_preds})
submission.to_csv('submission.csv', index=False)
print(f"Submission salva!")
print(submission['target'].value_counts().sort_index())